In [1]:
import sympy as sp, numpy as np, matplotlib.pyplot as plt

## Rewrtiting clockwork action

In [9]:
x = sp.symbols("x")
A = sp.cos(sp.atan(x) * 2)
A.expand(trig=True)

-1 + 2/(x**2 + 1)

In [10]:
A = sp.sin(sp.atan(x) * 2)
A.expand(trig=True)

2*x/(x**2 + 1)

## Integrate y direction out in clockwork

In [2]:
R, m, y, phi = sp.symbols("R, m, y, phi")  
I = 4 * sp.exp(2*m*y) / (1 + sp.exp(2*m*y) * phi**2)**2
ans = sp.integrate(I, (y, 0, sp.pi*R))
ans.simplify()

(2*exp(2*pi*R*m) - 2)/(m*(phi**2 + 1)*(phi**2*exp(2*pi*R*m) + 1))

## approximation of coth

In [17]:
epsilon = sp.symbols("epsilon")

In [18]:
sp.series((1 + epsilon**2) / (1 - epsilon**2), epsilon, 0, 3)

1 + 2*epsilon**2 + O(epsilon**3)

## Pertubation of equation of state in tracking solutions

In [4]:
w, delta_w = sp.symbols("w, delta_w")

In [5]:
x = (1 - w) / (1 + w)

In [6]:
x.subs(w, w + delta_w).series(delta_w, 0, 2).simplify()

1/(w + 1) - w/(w + 1) - 2*delta_w/(w + 1)**2 + O(delta_w**2)

In [8]:
y = sp.symbols("y")

In [9]:
S = 1 / (y - 2 / (w + 1)**2 * delta_w)
S.series(delta_w, 0, 2).subs(y, x)

(w + 1)/(1 - w) + 2*delta_w/(1 - w)**2 + O(delta_w**2)

In [10]:
(1 / x.subs(w, w + delta_w)).series(delta_w, 0, 2).simplify()

-1/(w - 1) - w/(w - 1) + 2*delta_w/(w - 1)**2 + O(delta_w**2)

In [11]:
a = sp.symbols("a")

In [19]:
S = sp.sqrt((1 - sp.log(a)*3*delta_w) / (1 + w + delta_w))
S.series(delta_w, 0, 2).simplify()

((w + 1)**(3/2) - delta_w*sqrt(w + 1)*(3*(w + 1)*log(a) + 1)/2 + O(delta_w**2))/(w + 1)**2

## multi axions

In [9]:
n = 2
Q = np.array([[sp.Symbol(f"Q_{i}{j}", real=True) for j in range(n)] for i in range(n)])
phi = [sp.Symbol(f"phi_{i}", real=True) for i in range(n)]
f = [sp.Symbol(f"f_{i}", positive=True) for i in range(n)]
Lambda = [sp.Symbol(f"Lambda_{i}", positive=True) for i in range(n)]
V = sum(Lambda[i]**4 * (1 - sp.cos(sum(Q[i, j] * phi[j] / f[j] for j in range(n)))) for i in range(n))
V

Lambda_0**4*(1 - cos(Q_00*phi_0/f_0 + Q_01*phi_1/f_1)) + Lambda_1**4*(1 - cos(Q_10*phi_0/f_0 + Q_11*phi_1/f_1))

In [27]:
def my_series(f, x, x0, n):
    return sum(sp.diff(f, x, i).subs(x, x0) / sp.factorial(i) * (x - x0)**i for i in range(n)) 
V_approx = V
for i in range(n):
    V_approx = my_series(V_approx, phi[i], 0, 3)
V_approx = V_approx.simplify().expand()

In [31]:
print(sp.latex(V_approx))

- \frac{\Lambda_{0}^{4} Q_{00}^{2} Q_{01}^{2} \phi_{0}^{2} \phi_{1}^{2}}{4 f_{0}^{2} f_{1}^{2}} + \frac{\Lambda_{0}^{4} Q_{00}^{2} \phi_{0}^{2}}{2 f_{0}^{2}} + \frac{\Lambda_{0}^{4} Q_{00} Q_{01} \phi_{0} \phi_{1}}{f_{0} f_{1}} + \frac{\Lambda_{0}^{4} Q_{01}^{2} \phi_{1}^{2}}{2 f_{1}^{2}} - \frac{\Lambda_{1}^{4} Q_{10}^{2} Q_{11}^{2} \phi_{0}^{2} \phi_{1}^{2}}{4 f_{0}^{2} f_{1}^{2}} + \frac{\Lambda_{1}^{4} Q_{10}^{2} \phi_{0}^{2}}{2 f_{0}^{2}} + \frac{\Lambda_{1}^{4} Q_{10} Q_{11} \phi_{0} \phi_{1}}{f_{0} f_{1}} + \frac{\Lambda_{1}^{4} Q_{11}^{2} \phi_{1}^{2}}{2 f_{1}^{2}}


In [42]:
M = sp.Matrix([[V_approx.diff(phi[i]).diff(phi[j]).subs(phi[0], 0).subs(phi[1], 0).simplify() * 2 
            for j in range(n)] for i in range(n)])
M

Matrix([
[       2*(Lambda_0**4*Q_00**2 + Lambda_1**4*Q_10**2)/f_0**2, 2*(Lambda_0**4*Q_00*Q_01 + Lambda_1**4*Q_10*Q_11)/(f_0*f_1)],
[2*(Lambda_0**4*Q_00*Q_01 + Lambda_1**4*Q_10*Q_11)/(f_0*f_1),        2*(Lambda_0**4*Q_01**2 + Lambda_1**4*Q_11**2)/f_1**2]])

In [44]:
EV = M.eigenvects()

In [107]:
S = [f[1]**2, f[0]**2] @ (Q.T**2 @ np.array(Lambda)**4)
S

f_0**2*(Lambda_0**4*Q_01**2 + Lambda_1**4*Q_11**2) + f_1**2*(Lambda_0**4*Q_00**2 + Lambda_1**4*Q_10**2)

In [117]:
eigen_vals = sp.Matrix([eigval.expand() for eigval, _, _ in EV])
R = eigen_vals * f[0]**2 * f[1]**2 - sp.Matrix([S,  S])
R.simplify()
R2 = R[0]**2
R3 = R2.factor()

In [130]:
R

Matrix([
[-sqrt(Lambda_0**8*Q_00**4*f_1**4 + 2*Lambda_0**8*Q_00**2*Q_01**2*f_0**2*f_1**2 + Lambda_0**8*Q_01**4*f_0**4 + 2*Lambda_0**4*Lambda_1**4*Q_00**2*Q_10**2*f_1**4 - 2*Lambda_0**4*Lambda_1**4*Q_00**2*Q_11**2*f_0**2*f_1**2 + 8*Lambda_0**4*Lambda_1**4*Q_00*Q_01*Q_10*Q_11*f_0**2*f_1**2 - 2*Lambda_0**4*Lambda_1**4*Q_01**2*Q_10**2*f_0**2*f_1**2 + 2*Lambda_0**4*Lambda_1**4*Q_01**2*Q_11**2*f_0**4 + Lambda_1**8*Q_10**4*f_1**4 + 2*Lambda_1**8*Q_10**2*Q_11**2*f_0**2*f_1**2 + Lambda_1**8*Q_11**4*f_0**4)],
[ sqrt(Lambda_0**8*Q_00**4*f_1**4 + 2*Lambda_0**8*Q_00**2*Q_01**2*f_0**2*f_1**2 + Lambda_0**8*Q_01**4*f_0**4 + 2*Lambda_0**4*Lambda_1**4*Q_00**2*Q_10**2*f_1**4 - 2*Lambda_0**4*Lambda_1**4*Q_00**2*Q_11**2*f_0**2*f_1**2 + 8*Lambda_0**4*Lambda_1**4*Q_00*Q_01*Q_10*Q_11*f_0**2*f_1**2 - 2*Lambda_0**4*Lambda_1**4*Q_01**2*Q_10**2*f_0**2*f_1**2 + 2*Lambda_0**4*Lambda_1**4*Q_01**2*Q_11**2*f_0**4 + Lambda_1**8*Q_10**4*f_1**4 + 2*Lambda_1**8*Q_10**2*Q_11**2*f_0**2*f_1**2 + Lambda_1**8*Q_11**4*f_0**4)]]

In [122]:
R3._args[0]

Lambda_0**4*Q_00**2*f_1**2 + Lambda_0**4*Q_01**2*f_0**2 - 2*Lambda_0**2*Lambda_1**2*Q_00*Q_11*f_0*f_1 + 2*Lambda_0**2*Lambda_1**2*Q_01*Q_10*f_0*f_1 + Lambda_1**4*Q_10**2*f_1**2 + Lambda_1**4*Q_11**2*f_0**2

In [123]:
R3._args[1]

Lambda_0**4*Q_00**2*f_1**2 + Lambda_0**4*Q_01**2*f_0**2 + 2*Lambda_0**2*Lambda_1**2*Q_00*Q_11*f_0*f_1 - 2*Lambda_0**2*Lambda_1**2*Q_01*Q_10*f_0*f_1 + Lambda_1**4*Q_10**2*f_1**2 + Lambda_1**4*Q_11**2*f_0**2

In [126]:
D = (R3._args[0] - R3._args[1]).simplify()
D

4*Lambda_0**2*Lambda_1**2*f_0*f_1*(-Q_00*Q_11 + Q_01*Q_10)

In [129]:
(R3._args[0] - D/2).simplify()

Lambda_0**4*Q_00**2*f_1**2 + Lambda_0**4*Q_01**2*f_0**2 + Lambda_1**4*Q_10**2*f_1**2 + Lambda_1**4*Q_11**2*f_0**2

## reheating

In [2]:
t, t_0, t_prime = sp.symbols("t, t_0, t_prime", positive=True)

In [3]:
sp.integrate(1/t_prime**2 * sp.exp(-(t_prime - t_0)), (t_prime, t_0, t))

exp(t_0)*expint(2, t_0)/t_0 - exp(t_0)*expint(2, t)/t